In [233]:
import shutil
import numpy as np
import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import random

In [329]:
exist_res = torch.rand(8,19)
value_res = torch.randint(low=1, high=7, size=(8,19))
measure_x = exist_res.clone()
exist= 0.8
next_exist= 0.3
value = 2
print('initial')
print('exist:',exist_res)
print('value:',value_res)
print('x:',measure_x)

initial
exist: tensor([[0.7067, 0.0020, 0.5104, 0.9735, 0.0579, 0.8253, 0.3993, 0.2907, 0.7760,
         0.5661, 0.7981, 0.8189, 0.6965, 0.1594, 0.0811, 0.8909, 0.4188, 0.8473,
         0.5096],
        [0.8962, 0.7876, 0.1786, 0.0716, 0.4303, 0.8917, 0.6971, 0.9455, 0.8337,
         0.6808, 0.9288, 0.9116, 0.9560, 0.5887, 0.5197, 0.9193, 0.1199, 0.6607,
         0.5044],
        [0.2101, 0.4566, 0.2752, 0.7795, 0.2007, 0.3482, 0.8671, 0.6645, 0.5466,
         0.0917, 0.3789, 0.2624, 0.8560, 0.0081, 0.7736, 0.1274, 0.3733, 0.6043,
         0.6416],
        [0.1060, 0.5739, 0.5854, 0.8247, 0.7794, 0.1557, 0.5851, 0.5645, 0.2723,
         0.9888, 0.2388, 0.2661, 0.0223, 0.4454, 0.2186, 0.2204, 0.0837, 0.0271,
         0.3467],
        [0.0017, 0.6969, 0.4140, 0.3883, 0.3458, 0.9914, 0.8706, 0.6646, 0.3351,
         0.5059, 0.4328, 0.5842, 0.0626, 0.6805, 0.3359, 0.2886, 0.7416, 0.9681,
         0.0040],
        [0.0917, 0.0652, 0.0799, 0.2733, 0.0344, 0.8292, 0.7665, 0.7141, 0.7134,
    

In [ ]:
def exist_test(vector):
    random_list = []
    random_list = [element +random.random()-0.5 for element in vector]
    # [element / n for element in compare_vector]
    print("generate exist", random_list)
    return random_list

def value_test(vector):
    #random_list = [torch.randint(low=0, high=7), torch.randint(low=0, high=7)]
    random_list = [element*0 + random.randint(1,7) for element in vector]
    #random_list = torch.randint_like(vector, 0, 7)

    
    print("generate value", random_list)
    return random_list

In [341]:


def combine(i,j, count, exist_res, measure_x):
    compare_list = []
    compare_vector = torch.zeros_like(measure_x[i])
    new_exist_res = torch.tensor(0)
    new_value_res = torch.tensor(0)
    for n in range(count):
        compare_list.append([exist_res[i+m][j] for m in range(count)])
    
    print("combine start for ", count)
    print("compare vector initial ", compare_vector)
    print("compare list initial ", compare_list)
    
    for k in range(2,count+1):
        n=0
        # summation of embedding vector from measure i to measure i+k
        for l in range(k):
            compare_vector = compare_vector + measure_x[i+l]
            print('idx:',i+l)
            print(measure_x[i+l])
            print(compare_vector)
            n += 1
        # average of embedding vector from measure i to measure i+k
        compare_vector = compare_vector / n
        print('compare_count:',n)
        print('compare_x:',compare_vector)
        #get exist with compare_vector and append to compare_list
        exist = exist_test(compare_vector)
        compare_vector = torch.zeros_like(measure_x[i])
        for l in range(k):
            compare_list[k-1][l] = exist[j]
    

    print('compare_list:', compare_list)

    # Find the largest index among the rows with the highest mean 
    averages = [torch.mean(torch.stack(row)).item() for row in compare_list]  

    max_index = len(averages) - 1 - averages[::-1].index(max(averages))

    count = max_index + 1
    new_exist_res = compare_list[max_index][0]
    print("max:", max_index)
    print("new exist:", new_exist_res)
        
    
    for m in range(count):
        compare_vector = compare_vector + measure_x[i+l]
            
    compare_vector = compare_vector / n
    print('compare_count:',count)
    print('compare_x:',compare_vector)
    value = value_test(compare_vector)
    compare_vector = torch.zeros_like(measure_x[i])

    new_value_res = value[j]
    
    return count, new_exist_res, new_value_res
                
                
def get_segment(exist_res, value_res, measure_x, exist_thres, next_exist_thres, value_thres):
    # num of feature
    for j in range(exist_res.size(1)):
        i =0
        
        # search of measure        
        while i < exist_res.size(0):
            count = 1
            print("i:", i)
            
            if exist_res[i,j] > exist_thres and i < exist_res.size(0)-1:
                
                value = value_res[i,j]
                print(exist_res[i,j], value_res[i,j])
                # count represents number of consecutive measures that can potentially be combined with a pivot measure i
                for n in range (i+1, exist_res.size(0)):
                    print('n check', n)
                    print(exist_res[n,j], value_res[n,j])
                    if exist_res[n,j] > next_exist_thres and abs(value_res[n,j]-value) < value_thres:
                        count += 1
                        print('check')
                    else:
                        break
                                        
                if count > 1:
                    # compare existence of candidate measure set
                    # count: the length of segment start from measure i
                    # exist_res: reflecting the existence of combined segment
                    # value_res: reflecting the feauture value of combined segment
                    count, new_exist_res, new_value_res = combine(i,j, count, exist_res, measure_x)

                    for m in range(count):
                        exist_res[i+m][j] = new_exist_res
                        value_res[i+m][j] = new_value_res
            
            # give 0 to existence and feature value of measures whose existence does not exceed the threshold
            if exist_res[i,j] <= exist_thres:
                print("pass")
                print(i)
                exist_res[i,j] = torch.tensor(0)
                value_res[i,j] = torch.tensor(0)
            
            # Move to the next measure of the combined measure 
            i += count
            
    return exist_res, value_res
    

In [342]:
print(exist_res)
print(value_res)
print(measure_x)
exist_res, value_res = get_segment(exist_res, value_res, measure_x, exist, next_exist, value)
print('output')
print('exist:',exist_res)
print('value:',value_res)
print('x:',measure_x)

tensor([[0.0000, 0.0000, 0.0000, 0.9735, 0.0000, 0.8253, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.8189, 0.0000, 0.0000, 0.0000, 0.8909, 0.0000, 0.9470,
         0.0000],
        [0.8962, 0.0000, 0.0000, 0.0000, 0.0000, 0.8917, 0.0000, 1.2224, 0.9612,
         0.0000, 0.9288, 0.9116, 1.3698, 0.0000, 0.0000, 0.9193, 0.0000, 0.9470,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8671, 1.2224, 0.9612,
         0.0000, 0.0000, 0.0000, 1.3698, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.8247, 0.0000, 0.0000, 0.0000, 1.2224, 0.0000,
         1.1036, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9914, 0.8706, 0.0000, 0.0000,
         1.1036, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.9681,
         0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.8292, 0.0000, 0.0000, 0.0000,
         0.0000, 0.

In [321]:
measure_x

8